In [26]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [2]:
rd_df=pd.read_csv("DP_LIVE_26032021032725352.csv")
rd_df.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,GDEXPRD,TOT,MLN_USD,A,2000,10689.513545,NaN
1,AUS,GDEXPRD,TOT,MLN_USD,A,2002,12782.111023,NaN
2,AUS,GDEXPRD,TOT,MLN_USD,A,2004,14407.180094,NaN
3,AUS,GDEXPRD,TOT,MLN_USD,A,2006,17803.608913,NaN
4,AUS,GDEXPRD,TOT,MLN_USD,A,2008,21079.459211,NaN


In [3]:
namemap_df=pd.read_csv("country_WoS_ECI_WB_flags.tsv",sep="\t")
namemap_df.head()

,WoS,ECI_Country,WB,Code
0,Albania,Albania,Albania,ALB
1,Algeria,Algeria,Algeria,DZA
2,Angola,Angola,Angola,AGO
3,Argentina,Argentina,Argentina,ARG
4,Australia,Australia,Australia,AUS


In [4]:
rd_df=rd_df.merge(namemap_df[['Code','WoS']],left_on='LOCATION',right_on='Code')
rd_df.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes,Code,WoS
0,AUS,GDEXPRD,TOT,MLN_USD,A,2000,10689.513545,NaN,AUS,Australia
1,AUS,GDEXPRD,TOT,MLN_USD,A,2002,12782.111023,NaN,AUS,Australia
2,AUS,GDEXPRD,TOT,MLN_USD,A,2004,14407.180094,NaN,AUS,Australia
3,AUS,GDEXPRD,TOT,MLN_USD,A,2006,17803.608913,NaN,AUS,Australia
4,AUS,GDEXPRD,TOT,MLN_USD,A,2008,21079.459211,NaN,AUS,Australia


In [23]:
yearlist=[2009,2015,2018]
ranklist=[2,5,10]
result_df=pd.DataFrame(index=ranklist,columns=yearlist)
for year in yearlist:
    rd_filter=rd_df[rd_df.TIME==year]
    rd_filter=rd_filter.sort_values(by='Value',ascending=False)
    for r in ranklist:
        total=rd_filter['Value'].sum()
        topr=rd_filter[:r]['Value'].sum()
        topr=topr/total
        result_df.loc[r,year]=topr

In [51]:
np.round(result_df,2)

,2009,2015,2018
2,0.482378,0.505921,0.546234
5,0.713291,0.716943,0.754844
10,0.851455,0.839711,0.871668


In [27]:
pubs_author_funder = pd.read_csv("..//..//data//dropbox//Data//DerivedData//CleanedRawData//pubs_author_funder.csv")
pubs_author_funder['author_country'] = pubs_author_funder['author_country'].apply(literal_eval)
pubs_author_funder['funder_country'] = pubs_author_funder['funder_country'].apply(
    lambda x: literal_eval(x) if pd.notnull(x) else "Not-Funded")
#pubs_author_funder['funding_units'] = pubs_author_funder['funder_country'].apply(lambda x: len(x))
pubs_author_funder.head()

,id,author_country,funder_country,discipline,year,rel_citation,citation
0,30507292,[Turkey],[Turkey],77,2009,0.485,6
1,30574690,"[United States, France]","[United States, United States, United States, ...",83,2009,0.703,9
2,30574691,[Russia],[Russia],83,2009,0.312,4
3,30585095,"[Spain, Spain]","[Spain, Spain, Spain, Spain, Spain, Spain]",89,2009,0.137,1
4,30585097,"[Germany, Spain]",[Spain],89,2009,0.410,3


In [ ]:
pubs_filter=pubs_author_funder[pubs_author_funder.funder_country!='Not-Funded']

In [40]:
cntry_frac=[]
for index, row in pubs_filter.iterrows():
    id=row['id']
    funder=list(set(row['funder_country']))
    year=row['year']
    n=len(funder)
    for f in funder:
        cntry_frac.append([id,f,1/n,year])

In [46]:
cntry_frac_df=pd.DataFrame(cntry_frac,columns=['id','cntry','count','year'])
cntry_frac_df.head()

,id,cntry,count,year
0,30507292,Turkey,1.0,2009
1,30574690,United States,1.0,2009
2,30574691,Russia,1.0,2009
3,30585095,Spain,1.0,2009
4,30585097,Spain,1.0,2009


In [47]:
cntry_frac_df=cntry_frac_df.groupby(['cntry','year'])['count'].sum().reset_index()
cntry_frac_df.head()

,cntry,year,count
0,Afghanistan,2010,1.000000
1,Afghanistan,2011,0.333333
2,Afghanistan,2012,0.333333
3,Afghanistan,2015,0.456960
4,Afghanistan,2016,0.160256


In [48]:
yearlist=[2009,2015,2018]
ranklist=[2,5,10]
result_pub=pd.DataFrame(index=ranklist,columns=yearlist)
for year in yearlist:
    cntry_filter=cntry_frac_df[cntry_frac_df.year==year]
    cntry_filter=cntry_filter.sort_values(by='count',ascending=False)
    for r in ranklist:
        total=cntry_filter['count'].sum()
        topr=cntry_filter[:r]['count'].sum()
        topr=topr/total
        result_pub.loc[r,year]=topr

In [49]:
result_pub

,2009,2015,2018
2,0.410961,0.444974,0.482898
5,0.555659,0.563697,0.594346
10,0.710344,0.714049,0.735693
